# Sử dụng model phobert-base cho bài toán regress tính khuynh hướng giá stock dựa trên tin tức

### Load data from google sheet 
https://docs.google.com/spreadsheets/d/1ZSLM2gc6GBWs1fvvNwzA0D7jusRWv2xPVqOVoXcYJ3o/edit#gid=732292548

In [152]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('./key/questioniso-0a60ce8c39c1.json', scope)

# authorize the clientsheet 
client = gspread.authorize(creds)

sheet = client.open_by_key('1ZSLM2gc6GBWs1fvvNwzA0D7jusRWv2xPVqOVoXcYJ3o')

# get the first sheet of the Spreadsheet
sheet_instance = sheet.worksheet('news')

data = sheet_instance.get_all_records()
print('data', data[:1])

data [{'id': 1, 'title': 'Dow Jones tăng hơn 300 điểm, chấm dứt 5 phiên giảm liên tiếp', 'content': 'Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.\n\n\n\nKết thúc phiên giao dịch ngày thứ Ba, chỉ số Dow Jones tăng 336.26 điểm (tương đương 1.06%) lên 32,155.40 điểm, chấm dứt chuỗi 5 phiên lao dốc liên tiếp. Chỉ số S&P 500 tiến 1.65% lên 3,919.29 điểm. Chỉ số Nasdaq Composite cộng 2.14% lên 11,428.15 điểm.\n\nSự nhiệt tình mua vào các cổ phiếu ngân hàng của nhà đầu tư đã giảm đi phần nào vào buổi chiều. Tuy nhiên, nhiều cổ phiếu vẫn ghi nhận đà tăng, đánh dấu bước đảo chiều sau 2 phiên giảm sâu khi nhà đầu tư ngày càng tin tưởng rằng những ngân hàng đó sẽ không chịu chung số phận như ngân hàng Silicon Valley và Signature. Các nhà quản lý cho biết vào ngày 12/3 rằng họ đã xây dựng một kế hoạch để can thiệp cho tất cả những người gửi tiền ở 2 ngân hàng này.\

In [153]:
df = pd.DataFrame()
title = []
content = []
date = []
stock_price = []
status = []

for values in data:
    title.append(values['title'])
    content.append(values['content'])
    date.append(values['date'])
    stock_price.append(values['stock_price'])
    status.append(values['status'])

df['title'] = title
df['content'] = content
df['date'] = date
df['stock_price'] = stock_price
df['status'] = status

df[0:5]

,title,content,date,stock_price,status
0,"Dow Jones tăng hơn 300 điểm, chấm dứt 5 phiên ...",Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3)...,2023/02/07,100,1.0
1,Vàng thế giới lùi nhẹ khi lợi suất trái phiếu ...,Giá vàng giảm vào ngày thứ Ba (14/3) khi lợi s...,2023/02/08,98,-0.5
2,Xét xử vụ DongABank: 'Lạnh gáy' với khoản vay ...,Sau khi bị cáo Phùng Ngọc Khánh (cựu Chủ tịch ...,2023/02/09,95,-1.0
3,Cựu tổng giám đốc Ngân hàng Đông Á Trần Phương...,Cùng các đồng phạm gây thiệt hại tổng số tiền ...,2023/02/10,93,-1.0
4,Nóng: Ngân hàng Nhà nước giảm lãi suất điều hành,Ngân hàng Nhà nước (NHNN) vừa ban hành các quy...,2023/02/11,94,1.0


In [154]:
import pandas as pd
from datetime import date, timedelta
import re
from underthesea import word_tokenize, sent_tokenize
from gensim import corpora, models

def clean_text(text):
    text = str(text).lower() # lowercase text
    return text

df = df[df['stock_price'] != '']

documents = df['content'].values.tolist()
print('documents:', documents[0:1])
print('len:', len(documents))

documents: ['Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.\n\n\n\nKết thúc phiên giao dịch ngày thứ Ba, chỉ số Dow Jones tăng 336.26 điểm (tương đương 1.06%) lên 32,155.40 điểm, chấm dứt chuỗi 5 phiên lao dốc liên tiếp. Chỉ số S&P 500 tiến 1.65% lên 3,919.29 điểm. Chỉ số Nasdaq Composite cộng 2.14% lên 11,428.15 điểm.\n\nSự nhiệt tình mua vào các cổ phiếu ngân hàng của nhà đầu tư đã giảm đi phần nào vào buổi chiều. Tuy nhiên, nhiều cổ phiếu vẫn ghi nhận đà tăng, đánh dấu bước đảo chiều sau 2 phiên giảm sâu khi nhà đầu tư ngày càng tin tưởng rằng những ngân hàng đó sẽ không chịu chung số phận như ngân hàng Silicon Valley và Signature. Các nhà quản lý cho biết vào ngày 12/3 rằng họ đã xây dựng một kế hoạch để can thiệp cho tất cả những người gửi tiền ở 2 ngân hàng này.\n\nChứng chỉ quỹ SPDR S&P Regional Banking ETF tiến 2%, phục hồi phần nào sau đà lao dốc

### Tính trung bình word2vec để tạo ra mảng training chỉ 2 chiều:
(dựa theo: https://github.com/PradipNichite/Youtube-Tutorials/blob/main/Yotutube_WordVectors.ipynb)

In [155]:
import numpy as np
y_train = df['status'].values.tolist()
statusNp = np.asarray(df['status'].values.tolist())
Y_train = np.expand_dims(statusNp, axis = 1)
print(Y_train.shape, Y_train)

(22, 1) [[ 1. ]
 [-0.5]
 [-1. ]
 [-1. ]
 [ 1. ]
 [-1. ]
 [-1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [-1. ]
 [ 0.9]
 [-1. ]
 [-1. ]
 [-1. ]
 [ 0.5]
 [ 0.1]
 [ 0.4]
 [ 0.2]
 [-0.3]
 [-0.2]]


In [156]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1)) 
print(scaler.fit(Y_train))
print(scaler.data_max_)

MinMaxScaler(feature_range=(-1, 1))
[1.]


In [157]:
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

Some weights of the model checkpoint at vinai/phobert-base-v2 were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens

In [158]:
print(np.zeros((2, 5)))

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [184]:
from torch.nn.functional import pad
def trim_words(sentence, num_word = 200):
    # Split the sentence into words
    words = sentence.split()
    # Trim the sentence to 5 words
    trimmed_sentence = " ".join(words[:num_word])
    return trimmed_sentence

max_len = 400
def vectorize(sentence):
    sentence_token = trim_words(word_tokenize(sentence, format="text"))
    input_ids = torch.tensor([tokenizer.encode(sentence_token)])
    features = phobert(input_ids)  # Models outputs are now tuples
    features_np = features.last_hidden_state.detach()
    padding_length = max_len - len(features_np[0])
    padded_a = pad(features_np, (0, 0, 0, padding_length), value=0, mode='constant')
    return padded_a.detach().numpy()[0]

def vectorize_arr(sentences):
    t = np.array([vectorize(sentence) for sentence in sentences])
    return t


X_train = vectorize_arr(documents)

documents: (400, 768)


In [188]:
text = """
Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.
Các nhà đầu tư đang thực hiện mua bán đầu cơ là nhiều.
"""

# (1, 239, 768)
print('documents:', vectorize(text).shape)

sentence_token = trim_words(word_tokenize(text, format="text"))
input_ids = torch.tensor([tokenizer.encode(sentence_token)])

words = []
for index in input_ids[0]:
    word = tokenizer.decode([index])
    words.append(word)
print('words', words)

features = phobert(input_ids)  # Models outputs are now tuples
print(input_ids, features)

documents: (400, 768)
words ['<s>', 'Chứng_khoán', 'Mỹ', 'khởi_sắc', 'vào', 'ngày', 'thứ@@', '_B@@', 'a', '(', '14/3', ')', ',', 'khi', 'nhà_đầu_tư', 'đặt_cược', 'vào', 'việc', 'nguy_cơ', 'lan_truyền', 'sau', 'vụ', 'phá_sản', 'của', 'các', 'ngân_hàng', 'Silicon_@@', 'Valley', 'và', 'Signa@@', 'ture', 'đã', 'được', 'ngăn_chặn', '.', 'Các', 'nhà_đầu_tư', 'đang', 'thực_hiện', 'mua_bán', 'đầu_cơ', 'là', 'nhiều', '.', '</s>']
tensor([[    0,  5149,    93,  7864,    33,    43, 26286,  2848,  1517,    20,
         18521,    19,     4,    26,   887, 13016,    33,    49,   750,  5384,
            53,   178,  5905,     7,     9,   610, 41963, 23442,     6, 50348,
         11371,    14,    11,  1608,     5,   146,   887,    52,   112,  1862,
          7625,     8,    36,     5,     2]]) BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0949,  0.5619, -0.1988,  ...,  0.1258,  0.3684,  0.1136],
         [-0.1720,  0.0289, -0.4192,  ...,  0.0990,  0.1530, -0.0903],
        

In [161]:
print(X_train.shape)

(22, 400, 768)


In [234]:
# Importing the necessary library
import numpy as np
import tensorflow as tf
from keras.layers import MaxPooling1D, BatchNormalization, Rescaling, Normalization, LayerNormalization, Lambda

# Defining the input array
input_array = np.array(
    [[[1], [2], [12]], [[4], [5], [6]], [[7], [8], [9]]]
    )

# Define a custom function or lambda expression
def custom_function(x):
        # Compute the average along the second axis
    average_tensor = tf.reduce_mean(x, axis=1)

    # Min-max scale the average tensor
    # min_val = tf.reduce_min(average_tensor)
    # max_val = tf.reduce_max(average_tensor)
    # scaled_tensor = 2 * ((average_tensor - min_val) / (max_val - min_val)) - 1

    # Print the scaled tensor
    # print(scaled_tensor)
    return average_tensor

# Create a Lambda layer
layer = Lambda(custom_function)

# Printing the output
print(layer(input_array))


tf.Tensor(
[[5]
 [5]
 [8]], shape=(3, 1), dtype=int64)


In [252]:
from keras.models import Sequential
from keras.layers import MaxPooling1D, BatchNormalization, Rescaling, Dropout, Flatten, Dense, LSTM, Bidirectional, GlobalAveragePooling1D, Input, Average, LayerNormalization

input_shape = (400, 768)
model = Sequential()
model.add(Input(shape=input_shape))
model.add(Bidirectional(LSTM(
        100, 
        return_sequences = True, 
        recurrent_dropout=0.2)))

model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
# model.add(Rescaling(
#         scale=1, offset=-1,
#         input_shape=(400, 1)
#         ))
model.add(Dense(1))
model.add(BatchNormalization())
model.add(GlobalAveragePooling1D())

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()
results = model.predict(X_train[:9])
print(results.shape, results[:2])

Model: "sequential_70"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_70 (Bidirecti  (None, 400, 200)         695200    
 onal)                                                           
                                                                 
 dense_100 (Dense)           (None, 400, 10)           2010      
                                                                 
 dropout_34 (Dropout)        (None, 400, 10)           0         
                                                                 
 dense_101 (Dense)           (None, 400, 1)            11        
                                                                 
 batch_normalization_32 (Bat  (None, 400, 1)           4         
 chNormalization)                                                
                                                                 
 global_average_pooling1d_47  (None, 1)              

In [253]:

model.fit(X_train, Y_train, epochs=10, batch_size=10)

Epoch 1/10
3/3 [==============================] - 5s 675ms/step - loss: 0.8007 - mae: 0.8199
Epoch 2/10
3/3 [==============================] - 2s 593ms/step - loss: 0.3066 - mae: 0.4743
Epoch 3/10
3/3 [==============================] - 2s 594ms/step - loss: 0.2661 - mae: 0.4620
Epoch 4/10
3/3 [==============================] - 2s 570ms/step - loss: 0.1947 - mae: 0.3536
Epoch 5/10
3/3 [==============================] - 2s 584ms/step - loss: 0.1846 - mae: 0.3251
Epoch 6/10
3/3 [==============================] - 2s 588ms/step - loss: 0.0934 - mae: 0.2641
Epoch 7/10
3/3 [==============================] - 2s 633ms/step - loss: 0.1240 - mae: 0.2736
Epoch 8/10
3/3 [==============================] - 2s 609ms/step - loss: 0.2768 - mae: 0.4592
Epoch 9/10
3/3 [==============================] - 2s 590ms/step - loss: 0.1242 - mae: 0.2993
Epoch 10/10
3/3 [==============================] - 2s 604ms/step - loss: 0.0926 - mae: 0.2489


result:
 [[-0.00193834]
 [-0.04076922]
 [ 0.09973867]
 [ 0.13607511]
 [ 0.02344097]
 [-0.00054041]]

 

In [254]:
test_data = [
    "Cùng các đồng phạm gây thiệt hại tổng số tiền trên 5.500 tỉ đồng, cựu tổng giám đốc Ngân hàng Đông Á Trần Phương Bình tiếp tục bị đề nghị truy tố trong vụ án thứ 4.",
    "Công ty Lọc hóa dầu Bình Sơn (BSR) lên kế hoạch bảo dưỡng lớn cho nhà máy lọc dầu Dung Quất, đề ra mức lãi giảm gần 90%.",
    "Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.",
    "Nhà đầu tư mới của Bamboo Airways sẽ hỗ trợ tài chính cho hãng và giúp ông Trịnh Văn Quyết một khoản tiền để khắc phục hậu quả vụ án (nếu có).",
    "Thị trường chứng khoán năm 2023 không hoàn toàn như vẻ ngoài chúng ta đang thấy.",
    "Chỉ cần loại bỏ nhóm doanh nghiệp này, tất cả đều là “gã khổng lồ” về công nghệ, chỉ số này sẽ chẳng đi đến đâu."
]

test_data_c = vectorize_arr(test_data)
# print('test_data_c:', test_data_c.shape)
result = model.predict(test_data_c)
print('result:\n', result)

# scaler = MinMaxScaler(feature_range=(-1, 1)) 
# print(scaler.fit(result))
# print(scaler.data_max_)
# print(scaler.transform(result))

# i have numpy array with shape (6, 400, 1), how to convert it to (6, 1) use max result from array ?
# result = np.amax(result, axis=1)
# print('result:\n', result)
# print(scaler.transform(result))

1/1 [==============================] - 0s 248ms/step
result:
 [[-0.02144255]
 [-0.038692  ]
 [ 0.12251861]
 [ 0.1090144 ]
 [ 0.0224402 ]
 [-0.00058876]]
